In [1]:
from __future__ import division, print_function, absolute_import

import os
import numpy as np
import tensorflow as tf
import logging
from data_transformer.data_formatter import DataFormatter
from data_transformer.preprocess import Preprocessing

from data_transformer.data_prep import DataIO, genDistinctStratifiedBatches, genRandomStratifiedBatches
from nn.load_params import layer_name, convShape, getWeights
from nn.utils import getTriplets, tripletLoss
from train_test.model import *
from train_test.classify import SVM
from config import path_dict

logging.basicConfig(level=logging.DEBUG, filename="logfile.log", filemode="w",
                    format="%(asctime)-15s %(levelname)-8s %(message)s")

training = False
verification = False
create_batches = False

# ONE TIME RUN: 
------------
#### INPUT :  Folder path with images of several people, ensure the image folders are named with the person name
#### OUTPUT: Dumps a pickle file with three keys, dataX, dataY, labelDict. 
            * dataX: images converted into nd array
            * dataY: for each record of nd array, Labels are numerical (1,2,3,4,5)
            * labelDict: Contains the label corresponding to person name.



In [2]:
if training:
    objDP = DataFormatter(path_dict['parent_path'], 'training')
    objDP.createResizedData()
    dataX, dataY, labelDict = objDP.imageToArray()
    DataFormatter.dumpPickleFile(dataX, dataY, labelDict,
                               folderPath=os.path.join(path_dict['data_model_path']),
                               picklefileName='training_imgarr.pickle')
if verification:
    objDP = DataFormatter(path_dict['parent_path'], 'verification')
    objDP.createResizedData()
    dataX, dataY, labelDict = objDP.imageToArray()
    DataFormatter.dumpPickleFile(dataX, dataY, labelDict,
                               folderPath=os.path.join(path_dict['data_model_path']),
                               picklefileName='verification_imgarr.pickle')

# CREATE RANDOM BATCHES:
----------------
#### INPUT: Image nd array as input  [num_images, imgX, imgY, num_channels]
#### OUTPUT: Outputs a pickle file with shape [num_batches, num_image_per_batch, imgX, imgY, num_channels]

       *  We would wanna do stocastic descent for minibatches and update the parameters perbatch. This module attempts to create stratified batches (each batch would have equal distribution of labels). 
       
       * when genDistinctStratifiedBatches. The images in the batched would be distinct (would not repeat)
       * when genRandomStratifiedBatches. No seed is set for shuffling. So Images in different batches may repeat.

In [3]:
if create_batches:
    trainX, trainY, trainLabelDict = DataIO.getPickleFile(path_dict['data_model_path'],
                                                                 'training_imgarr.pickle')
    verX, verY, verLabelDict = DataIO.getPickleFile(path_dict['data_model_path'],
                                                           'verification_imgarr.pickle')
    print(trainX.shape, trainY.shape)
    print(verX.shape, verY.shape)
    genDistinctStratifiedBatches(trainX, trainY,
                          fileName='distinct_stratified_batches.pickle')

## RESET TENSORFLOW GRAPH

In [4]:
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

## GET INCEPTION WEIGHTS

In [5]:
moduleWeightDict = getWeights(path_dict['inception_nn4small_weights_path'])

# TRAIN AND TEST
---------

TO DO's:

1. Remove the random weight initialiazer for the last layer, and initialize it 
   with the inception net weights.  **DONE**

2. implement a module to save weights as checkpoints to the disk.  **DONE**

3. create a function to toggle between Random weight initializer, Inception net weight initializer 
   and using the saved checkpoint for the last Inception layer. **DONE**
   
4 : REMEBER TO STORE THE exponential weighted average of mean and variable in the batch normalization 
      fine tune function. SET THESE AS A VARIABLE (LOOK AT CIFAR CODE FOR HELP) **DONE**
 
5. Add more images.

6. Create a complete workflow train the network and perform cross validation: **DONE**

7. Store 1 image encodings for the 3-4 labels you have.

8. For a new image, pass the image throught network, get the encoding and see which is the most closest face using the encoding from the step 6.

9. Try :
    1. SVM classfication on embedding feature space: Get cross validation accuracy: **DONE**
    2. Softmax classification on embedding feature space: Get cross validation accuracy. 
    
10. The triplet selection now has, random selection of Hard negative. Having random selection makes it difficult to adjust parameters. So make is generated by a sedd, but the sees itself should be generated randomly via a different sees. Since having the same seeed decide a triplet would be problematic becasue the same hard negative would always be selected. **DONE**

11. Add learning rate decay.

In [6]:
# from __future__ import division, print_function, absolute_import

# import os
# import numpy as np
# from sklearn.svm import SVC
# from sklearn.externals import joblib

# class SVM():
#     '''
#     # The embeddings in a nutshell are features. The face image goes through a complex network and results in
#     # embeddings that captures complex features of a face. SVM's are good at classifying small datasets.
#     # SVM are also robust to over fitting. The idea here is that we would wanna learn a SVM classifier using the
#     # embeddings as the feature space and see for the given embedding, how many times we are able to predict the
#     # correct class
#     '''
    
#     def __init__(self):
#         pass
    
#     def train(self, embeddings, labels, model_name=None):
#         '''
#         :param embeddings:   Embeddings of the image
#         :param labels:       labels
#         :return:
#         '''
#         model = SVC(kernel='linear', probability=True)
#         model.fit(embeddings, labels)
#         joblib.dump(model, os.path.join(path_dict['classification_model_path'], str(model_name)+"_svm.sav"))
        
#     def classify(self, embeddings, model_name=None):
#         '''
#         :param embeddings: Image embeddings to classify
#         :param model_name:
#         :return:
#         '''
#         model = joblib.load(os.path.join(path_dict['classification_model_path'], str(model_name)+"_svm.sav"))
#         predLabels = model.predict_proba(embeddings)
#         top_label_idx = np.argmax(predLabels, axis=1)
#         labelProb = predLabels[np.arange(len(top_label_idx)), top_label_idx]
#         return top_label_idx, labelProb

In [10]:
from tensorflow.python.framework import ops
import config
from config import myNet

which_file = 'distinct_stratified_batches.pickle'
checkpoint_file_name = 'distinct_stratified_model'


'''
dataX = [num_batches, image_per_batch, image_x, image_y, image_channels]
dataY = [num_batches, labels]

'''

class Execute():
    def __init__(self, params, myNet, embeddingType='finetune'):
        self.params = params
        self.embeddingType = embeddingType
        self.myNet = myNet
        self.myNet['learning_rate'] = 0.0001
        
    def runPreprocessor(self, dataIN, sess):
        preprocessedData = np.ndarray(shape=(dataIN.shape), dtype='float32')
        for numImage in np.arange(dataIN.shape[0]):
            feed_dict = {
                self.preprocessGraphDict['imageIN']:dataIN[numImage,:]
            }
            preprocessedData[numImage,:] = sess.run(self.preprocessGraphDict['imageOUT'],
                                                      feed_dict=feed_dict)
        return preprocessedData
        
    def setNewWeights(self, sess):
        logging.info('UPDATING WEITHGS WITH FINETUNED WEIGHTS .........')
#         trainableVars = tf.get_collection(ops.GraphKeys.TRAINABLE_VARIABLES)
        if self.embeddingType=='finetune':
            for learned_vars in config.finetune_variables:
                scope, name = learned_vars.split(':')[0].split('/')
                if len(self.params[scope][name]) != 0:
                    var_ = sess.run(learned_vars)
                    logging.info('Updating param with scope %s and name %s and shape %s with shape %s',
                                 str(scope), str(name), str(self.params[scope][name].shape), str(var_.shape))
                    self.params[scope][name] = var_
                else:
                    raise ValueError('It seems that the scope %s or variable %s didnt exist in the dictionary ' % (str(scope), str(name)))
    
    def train(self, trnX_, trnY_, sess):
        '''
            1. Make the use of getEmbedding to get the graph with last layer parameter updated with the 
            fine tuned weights.
            2. Get the new embedding for batch/epoch using the computation graph
            3. Use the embeddings as feature for a classifier (svm/softmax)
            4. Classify faces using the new embeddings.
        '''
        trainEmbedGraph = getEmbeddings(self.myNet['image_shape'], self.params)
        embeddings = sess.run(trainEmbedGraph['output'], 
                              feed_dict={trainEmbedGraph['inpTensor']:trnX_})
        logging.info('Training Embeddings shape %s', embeddings.shape)
        obj_svm = SVM()
        obj_svm.train(embeddings, labels=trnY_, 
                      model_name='nFold_%s_batch_%s'%(str(self.nFold),str(self.epoch)))
        train_labels, train_label_prob = obj_svm.classify(embeddings, 
                                             model_name='nFold_%s_batch_%s'%(str(self.nFold),str(self.epoch)))
        return train_labels, train_label_prob
    
    def cvalid(self, cvX_, sess):
        embedGraph = getEmbeddings(self.myNet['image_shape'], self.params)
        embeddings = sess.run(embedGraph['output'], 
                              feed_dict={embedGraph['inpTensor']:cvX_})
        logging.info('Cross validation Embeddings shape %s', embeddings.shape)
        obj_svm = SVM()
        cv_labels, cv_label_prob = obj_svm.classify(embeddings, 
                                             model_name='nFold_%s_batch_%s'%(str(self.nFold),str(self.epoch)))
        return cv_labels, cv_label_prob
    
    def accuracy(self, y, y_hat):
        return np.mean(np.equal(y_hat, y))
    
#     def test(self, tstGraph, testBatch, sess):
#         # METHOD 2: TO get weights is form of Tensors
#         a = saver.restore(sess, os.path.join(checkpoint_path, "model.ckpt"))
#         trainableVars = tf.get_collection(ops.GraphKeys.TRAINABLE_VARIABLES)
#         testDict = getFineTunedEmbeddings([96,96,3], moduleWeightDict, trainableVars, sess)
#         embeddings = sess.run([tstGraph['output']], feed_dict={'inpTensor':testBatch})
#         return embeddings

    def run(self):
        # GET THE BATCH DATA FROM THE DISK
        dataX, dataY, labelDict = DataFormatter.getPickleFile(
            folderPath=path_dict['batchFolderPath'], picklefileName=which_file, getStats=True
        )
        trnBatch_idx = [list(np.setdiff1d(np.arange(len(dataX)), np.array(i))) for i in  np.arange(len(dataX))]
        cvBatch_idx = [i for i in  np.arange(len(dataX))]
        logging.info('dataX.shape = %s, dataY.shape = %s',str(dataX.shape), str(dataY.shape))

        # Reset graph to do a fresh start
        reset_graph()
        trn_embed_graph = trainEmbeddings(moduleWeightDict,init_wght_type='random')
        self.preprocessGraphDict = Preprocessing().preprocessImageGraph(
                                                            imageShape=self.myNet["image_shape"])
        # add ops to save and restore model
        saver = tf.train.Saver()

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            checkpoints = [ck for ck in os.listdir(path_dict['checkpoint_path']) if ck!='.DS_Store']
            if len(checkpoints) > 0 and self.myNet['use_checkpoint']:
                saver.restore(sess, os.path.join(path_dict['checkpoint_path'], 
                                                 "distinct_stratified_model.ckpt"))

            # LOOP FOR N-FOLD CROSS VALIDATION
            for nFold, (trn_batch_idx, cv_batch_idx) in enumerate(zip(trnBatch_idx, cvBatch_idx)):
                self.nFold = nFold + 1
                logging.info('RUNNING : %s FOLD ...........................', str(self.nFold))
                trnX = dataX[trn_batch_idx,:]
                trnY = dataY[trn_batch_idx,:]
                cvX = dataX[cv_batch_idx,:]
                cvY = dataY[cv_batch_idx,:]
                logging.info('trnX.shape = %s, trnY.shape = %s, cvX.shape = %s, cvY.shape = %s', 
                      str(trnX.shape), str(trnY.shape), str(cvX.shape), str(cvY.shape))

                for epoch in np.arange(10):
                    self.epoch = epoch + 1
                    logging.info('RUNNING : %s EPOCH ........................', str(self.epoch))
                    # Below loop will minimize the triplet loss and update the parameters
                    for batchNum, batchX in enumerate(trnX[0:len(trnX),:]):
                        logging.info('RUNNING BATCH %s for shape = %s', str(batchNum + 1), str(batchX.shape))
                        
                        # Step1 : Preprocess the Data
                        preprocessedData = self.runPreprocessor(dataIN=batchX, sess=sess)
                            
                        # Since we improve on our previous prediction, there can be cases where the network has learned a good enough
                        # decision boundary (for a batch) and is unable to find hard negative for the triplet selection. In such a case
                        # the network would return an empty array, which would raise a run time exception during the graph is computed.
                        # For such cases we would except an exception, and let the graph proceed. 
                        try:
                            opt, batch_loss, lr = sess.run([trn_embed_graph['optimizer'], 
                                                        trn_embed_graph['loss'],
                                                        trn_embed_graph['learning_rate']], 
                                                        feed_dict={trn_embed_graph['inpTensor']:preprocessedData})
                            print ('asasasasasas ', lr)
                        except Exception:
                            logging.info('Exception Raised! Check the log file and confirm if the exception is becasue of empty triplet array. If not then debugg it :)')       
                            logging.info("Fold = %s, Epoch = %s, Loss = %s", 
                                         str(self.nFold), str(self.epoch), "{:.6f}".format(batch_loss))
                            
                        print("Fold: " + str(self.nFold) + 
                              ", Epoch= " + str(self.epoch) + 
                              ", Loss= " + "{:.6f}".format(batch_loss))
#                     self.myNet['learning_rate'] = float(self.myNet['learning_rate']/2)
                    
                    save_path = saver.save(sess, os.path.join(path_dict['checkpoint_path'], "distinct_stratified_model.ckpt"))
                    

                    # Now that we have updated our parameters (weights and biases), we would
                    # fetch the embeddings using the updated parameter and train-test model
                    # to get an accuracy. Accuracy per epoch is now a good way to go
                    self.setNewWeights(sess) # replace the last layer's inception weights with leared finetuned weights
                    
                    # TRAIN, GET TRAINING PREDICTION AND ACCURACY
                    trnX_ = trnX.reshape(-1, trnX.shape[2], trnX.shape[3], trnX.shape[4]) # accumulate all batches
                    trnY_ = trnY.flatten()
                    train_labels, _ = self.train(trnX_, trnY_, sess)
                    tr_acc = self.accuracy(y=trnY_, y_hat=train_labels)
                    print (tr_acc)
                    
                    # GET CROSS VALIDATION PREDICTION AND ACCURACY
                    cv_labels, _ = self.cvalid(cvX, sess)
                    cv_acc = self.accuracy(y=cvY, y_hat=cv_labels)
                    print (cv_acc)
                    
#                     break

                break

objExec = Execute(params=moduleWeightDict, myNet=myNet, embeddingType='finetune')
objExec.run()

The shape of input data (X) is:  (10, 30, 96, 96, 3)
The shape of input data (Y) is:  (10, 30)
Unique labels in dataY is:  [ 0.  1.  2.]
Label dict:  None


/Users/sam/App-Setup/anaconda/envs/anaconda35/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


asasasasasas  0.0001
Fold: 1, Epoch= 1, Loss= 28.225758
asasasasasas  0.0001
Fold: 1, Epoch= 1, Loss= 25.516062
asasasasasas  0.0001
Fold: 1, Epoch= 1, Loss= 27.511787
asasasasasas  0.0001
Fold: 1, Epoch= 1, Loss= 27.326771
asasasasasas  0.0001
Fold: 1, Epoch= 1, Loss= 24.080223
asasasasasas  0.0001
Fold: 1, Epoch= 1, Loss= 25.608582
asasasasasas  0.0001
Fold: 1, Epoch= 1, Loss= 24.806387
asasasasasas  0.0001
Fold: 1, Epoch= 1, Loss= 25.493343
asasasasasas  0.0001
Fold: 1, Epoch= 1, Loss= 26.326994
0.774074074074
0.633333333333
asasasasasas  9.5e-05
Fold: 1, Epoch= 2, Loss= 21.787016
asasasasasas  9.5e-05
Fold: 1, Epoch= 2, Loss= 24.927269
asasasasasas  9.5e-05
Fold: 1, Epoch= 2, Loss= 24.518860
asasasasasas  9.5e-05
Fold: 1, Epoch= 2, Loss= 19.129396
asasasasasas  9.5e-05
Fold: 1, Epoch= 2, Loss= 23.473333
asasasasasas  9.5e-05
Fold: 1, Epoch= 2, Loss= 20.686478
asasasasasas  9.5e-05
Fold: 1, Epoch= 2, Loss= 23.580938
asasasasasas  9.5e-05
Fold: 1, Epoch= 2, Loss= 22.610264
asasasasas

# ANALYSIS SUMMARY:
-------------

For any deep learning problem, it is important to have more and more data. Our analysis, infact shows that this is true. Additionally, other stuffs like Exponential weighted average Batch normalization makes the model more robust.

* First we try to finetune the last layer of inception net with 90 images (30 image each label) and without exp weighted batch normalization. The accuracy of the model was very poor, jumping between 10% to 60%.

* Then we added more data 180 images (+30 images each label) the model performed little better than the previous model. But still the outcomes were inconsistent.

* Adding Exponential batch norm , made the results consistent accross runs. But we still had accuracy droping to 10%-20% per batch.

* We added more data 300 images (100 images per label). The results were far batter than the previous run and were also consistent. The results at this point were consistent. Cross validation 1 batch accuracy was between 60 to 70 percent. The problem however was that the accuracy decreased after epoch 6. Early stopping could help.

* After adding learning rate decay, as expected the cross validation accuracy gets more consistent accross different batches and produces a accuracy of 83% at epoch 10

#### TO NOTE:

The triplet selection is differnt for every differnt run even after having seed. This could be because small changes in embedding may initiate different triplet seletion. Embedding can be different becasue we have many random preprocessing steps. 

# ROUGH
-----------------------

In [9]:
import tensorflow as tf
import numpy as np
np.random.seed(1)
tfdata = tf.cast(np.random.rand(1,1,3,5) + 10, dtype=tf.float32)
print (tfdata.get_shape().as_list())
print (tfdata.get_shape()[-1])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batchMean, batchVar = tf.nn.moments(tfdata, axes=[0,1,2], name="moments")
    print (tfdata.eval())
    print (batchMean.eval())
    print (batchMean.get_shape().as_list())
    print (batchVar.eval())
    print (batchVar.get_shape().as_list())

[1, 1, 3, 5]
5
[[[[ 10.41702175  10.72032452  10.00011444  10.30233288  10.14675617]
   [ 10.09233856  10.18626022  10.34556103  10.39676762  10.53881645]
   [ 10.41919422  10.68521976  10.20445251  10.87811756  10.02738762]]]]
[ 10.30951786  10.5306015   10.18337631  10.52573872  10.23765373]
[5]
[ 0.02358428  0.05949085  0.020111    0.06357152  0.0477244 ]
[5]


In [8]:
np.array([1,2,3,4])[-1]

4

In [17]:

# reset_graph()
# def my_func(x):
#     return [x]

# inp = tf.placeholder(tf.int64)
# inp2 = tf.placeholder(tf.float)
# y = tf.py_func(my_func, [inp], tf.int64)
# y_1 = tf.add(y[:,0], 1)
# # print (len(y))#.get_shape())

# # a = np.array([[1,2,3],[4,5,6],[6,7,8]])
# a = []
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     try:
#         yy = sess.run([y_1],feed_dict={inp:a})
#         print (yy)
#     except InvalidArgumentError:
#         print ('dasdsdsdsdsdsdsds')
    

In [18]:
# from __future__ import print_function

# import tensorflow as tf

# # Import MNIST data
# from tensorflow.examples.tutorials.mnist import input_data

# reset_graph()
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# # Parameters
# learning_rate = 0.01
# training_epochs = 25
# batch_size = 100
# display_step = 1

# # tf Graph Input
# x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
# y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# # Set model weights
# W = tf.Variable(tf.zeros([784, 10]))
# b = tf.Variable(tf.zeros([10]))

# # Construct model
# pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# # Minimize error using cross entropy
# loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))


# global_step = tf.Variable(0, trainable=False)
# starter_learning_rate = 0.1
# # learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
# #                                            100, 0.96, staircase=True)
# learning_rate = tf.train.exponential_decay(starter_learning_rate,
#                                                    global_step * 500,  # Used for decay computation
#                                                    10000,  # Decay steps
#                                                    0.96,  # Decay rate
#                                                    staircase=True) 
# optimizer = (
#     tf.train.GradientDescentOptimizer(learning_rate)
#     .minimize(loss, global_step=global_step)
# )


# # Gradient Descent
# # optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# # Initialize the variables (i.e. assign their default value)
# init = tf.global_variables_initializer()

# # Start training
# with tf.Session() as sess:

#     # Run the initializer
#     sess.run(init)

#     # Training cycle
#     for epoch in range(training_epochs):
#         avg_cost = 0.
#         total_batch = int(mnist.train.num_examples/batch_size)
#         # Loop over all batches
#         for num, i in enumerate(range(total_batch)):
            
#             batch_xs, batch_ys = mnist.train.next_batch(batch_size)
# #             print (batch_ys)
#             # Run optimization op (backprop) and cost op (to get loss value)
#             _, c, l = sess.run([optimizer, loss, learning_rate], feed_dict={x: batch_xs,
#                                                           y: batch_ys})
#             print (epoch, num, l)
#             # Compute average loss
#             avg_cost += c / total_batch
#         print ('################')
#         # Display logs per epoch step
#         if (epoch+1) % display_step == 0:
#             print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

#     print("Optimization Finished!")

#     # Test model
#     correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
#     # Calculate accuracy
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))